# 57. 三角測量と3D復元
## Triangulation and 3D Reconstruction

---

## 学習目標

このノートブックを完了すると、以下ができるようになります：

- [ ] 三角測量の原理と幾何学的背景を理解する
- [ ] DLT（Direct Linear Transform）による三角測量を実装できる
- [ ] 中点法による三角測量を理解する
- [ ] 最適三角測量（Optimal Triangulation）の概念を理解する
- [ ] 多視点からの3D復元を実装できる
- [ ] 三角測量の誤差要因と精度向上の方法を理解する

---

## 前提知識

- 51: ピンホールカメラモデルと射影変換
- 53: 3D座標変換と剛体運動
- 55: エピポーラ幾何の理論
- 線形代数：SVD分解、最小二乗法

**難易度**: ★★★★☆（上級）  
**推定学習時間**: 90-120分

---

## 1. 三角測量とは

**三角測量（Triangulation）**は、複数の視点から観測した2D画像点の対応から、3D空間上の点の位置を復元する技術です。

### 基本原理

- 各画像点は、カメラ中心から3D点を通る**光線（ray）**に対応
- 2つの視点からの光線の**交点**が3D点の位置
- 理想的には2つの光線は1点で交わる
- 実際にはノイズにより交わらないため、**最良近似**を求める

### 応用

- ステレオビジョン
- Structure from Motion (SfM)
- SLAM
- モーションキャプチャ

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, List, Optional
from scipy.optimize import least_squares
import warnings
warnings.filterwarnings('ignore')

np.set_printoptions(precision=4, suppress=True)

print("ライブラリのインポート完了")

---

## 2. 三角測量の幾何学

### 2.1 問題設定

- **入力**:
  - 2つ以上のカメラの投影行列 $\mathbf{P}_i = \mathbf{K}_i [\mathbf{R}_i | \mathbf{t}_i]$
  - 各カメラでの画像点 $\mathbf{x}_i = (u_i, v_i)$

- **出力**:
  - 3D点 $\mathbf{X} = (X, Y, Z)$

### 2.2 投影方程式

$$\lambda_i \begin{pmatrix} u_i \\ v_i \\ 1 \end{pmatrix} = \mathbf{P}_i \begin{pmatrix} X \\ Y \\ Z \\ 1 \end{pmatrix}$$

### 2.3 理想と現実

- **理想**: 2つの光線が1点で交わる
- **現実**: ノイズにより光線は交わらない（ねじれの位置）

In [ ]:
def visualize_triangulation_geometry():
    """三角測量の幾何学を可視化"""
    fig = plt.figure(figsize=(14, 6))
    
    # 理想的なケース
    ax1 = fig.add_subplot(121, projection='3d')
    
    # カメラ中心
    C1 = np.array([0, 0, 0])
    C2 = np.array([1, 0, 0])
    
    # 3D点
    X_true = np.array([0.5, 0, 3])
    
    # 光線
    ax1.plot([C1[0], X_true[0]], [C1[1], X_true[1]], [C1[2], X_true[2]], 
             'b-', linewidth=2, label='Ray from Camera 1')
    ax1.plot([C2[0], X_true[0]], [C2[1], X_true[1]], [C2[2], X_true[2]], 
             'g-', linewidth=2, label='Ray from Camera 2')
    
    # カメラと点
    ax1.scatter(*C1, color='blue', s=200, marker='o', label='Camera 1')
    ax1.scatter(*C2, color='green', s=200, marker='o', label='Camera 2')
    ax1.scatter(*X_true, color='red', s=200, marker='*', label='3D Point')
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('Ideal Case: Rays Intersect', fontsize=12)
    ax1.legend()
    ax1.view_init(elev=20, azim=-60)
    
    # ノイズありのケース
    ax2 = fig.add_subplot(122, projection='3d')
    
    # ノイズを加えた方向
    noise = 0.05
    dir1 = X_true - C1
    dir1_noisy = dir1 + np.array([noise, -noise, 0])
    dir1_noisy = dir1_noisy / np.linalg.norm(dir1_noisy)
    
    dir2 = X_true - C2
    dir2_noisy = dir2 + np.array([-noise, noise, 0])
    dir2_noisy = dir2_noisy / np.linalg.norm(dir2_noisy)
    
    # 延長した光線
    t_range = 5
    ray1_end = C1 + t_range * dir1_noisy
    ray2_end = C2 + t_range * dir2_noisy
    
    ax2.plot([C1[0], ray1_end[0]], [C1[1], ray1_end[1]], [C1[2], ray1_end[2]], 
             'b-', linewidth=2, label='Noisy Ray 1')
    ax2.plot([C2[0], ray2_end[0]], [C2[1], ray2_end[1]], [C2[2], ray2_end[2]], 
             'g-', linewidth=2, label='Noisy Ray 2')
    
    # 最近点（中点法で推定）
    # 簡略化のため、光線間の最短距離の中点を計算
    def closest_points_on_rays(o1, d1, o2, d2):
        w0 = o1 - o2
        a = np.dot(d1, d1)
        b = np.dot(d1, d2)
        c = np.dot(d2, d2)
        d = np.dot(d1, w0)
        e = np.dot(d2, w0)
        
        denom = a * c - b * b
        if abs(denom) < 1e-10:
            return o1, o2
        
        s = (b * e - c * d) / denom
        t = (a * e - b * d) / denom
        
        p1 = o1 + s * d1
        p2 = o2 + t * d2
        
        return p1, p2
    
    p1, p2 = closest_points_on_rays(C1, dir1_noisy, C2, dir2_noisy)
    X_estimated = (p1 + p2) / 2
    
    # 最短距離を示す線分
    ax2.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 
             'r-', linewidth=2, linestyle='--', label='Shortest distance')
    
    # カメラと点
    ax2.scatter(*C1, color='blue', s=200, marker='o')
    ax2.scatter(*C2, color='green', s=200, marker='o')
    ax2.scatter(*X_true, color='red', s=200, marker='*', label='True 3D Point')
    ax2.scatter(*X_estimated, color='orange', s=200, marker='x', label='Estimated Point')
    
    ax2.set_xlabel('X')
    ax2.set_ylabel('Y')
    ax2.set_zlabel('Z')
    ax2.set_title('Real Case: Rays Do Not Intersect', fontsize=12)
    ax2.legend()
    ax2.view_init(elev=20, azim=-60)
    
    plt.tight_layout()
    plt.show()
    
    error = np.linalg.norm(X_estimated - X_true)
    print(f"真の3D点: {X_true}")
    print(f"推定された3D点: {X_estimated}")
    print(f"誤差: {error:.4f}")

visualize_triangulation_geometry()

---

## 3. DLT（Direct Linear Transform）による三角測量

### 3.1 基本原理

投影方程式から同次線形方程式系を導出し、SVDで解きます。

投影方程式：
$$\lambda \mathbf{x} = \mathbf{P} \mathbf{X}$$

展開すると：
$$\lambda u = \mathbf{p}_1^\top \mathbf{X}, \quad \lambda v = \mathbf{p}_2^\top \mathbf{X}, \quad \lambda = \mathbf{p}_3^\top \mathbf{X}$$

ここで $\mathbf{p}_i^\top$ は $\mathbf{P}$ の $i$ 行目。

### 3.2 拘束式の導出

$\lambda$ を消去：

$$u (\mathbf{p}_3^\top \mathbf{X}) - (\mathbf{p}_1^\top \mathbf{X}) = 0$$
$$v (\mathbf{p}_3^\top \mathbf{X}) - (\mathbf{p}_2^\top \mathbf{X}) = 0$$

### 3.3 線形方程式系

$$\mathbf{A} \mathbf{X} = \mathbf{0}$$

各カメラから2つの方程式が得られ、$n$ 視点で $2n \times 4$ の行列 $\mathbf{A}$ を構築。

In [ ]:
def triangulate_dlt(points_2d: List[np.ndarray], 
                    proj_matrices: List[np.ndarray]) -> np.ndarray:
    """DLT（Direct Linear Transform）による三角測量
    
    Args:
        points_2d: 各カメラでの2D点のリスト [(u1, v1), (u2, v2), ...]
        proj_matrices: 各カメラの投影行列のリスト [P1, P2, ...] (3x4)
    
    Returns:
        X: 3D点の座標 (3,)
    """
    n_views = len(points_2d)
    assert n_views >= 2, "少なくとも2視点が必要です"
    assert len(proj_matrices) == n_views
    
    # 行列Aの構築
    A = np.zeros((2 * n_views, 4))
    
    for i, (pt, P) in enumerate(zip(points_2d, proj_matrices)):
        u, v = pt[0], pt[1]
        
        # u * p3 - p1
        A[2*i] = u * P[2] - P[0]
        # v * p3 - p2
        A[2*i + 1] = v * P[2] - P[1]
    
    # SVDで解く
    U, S, Vt = np.linalg.svd(A)
    
    # 最小特異値に対応するベクトル
    X_homogeneous = Vt[-1]
    
    # 同次座標から3D座標へ
    X = X_homogeneous[:3] / X_homogeneous[3]
    
    return X

def create_projection_matrix(K: np.ndarray, R: np.ndarray, t: np.ndarray) -> np.ndarray:
    """投影行列 P = K [R | t] を作成"""
    Rt = np.hstack([R, t.reshape(-1, 1)])
    P = K @ Rt
    return P

def project_point(P: np.ndarray, X: np.ndarray) -> np.ndarray:
    """3D点を2D画像に投影"""
    X_h = np.append(X, 1)
    x_h = P @ X_h
    x = x_h[:2] / x_h[2]
    return x

print("DLT三角測量の実装完了")

In [ ]:
# テスト：合成データでDLTを検証

# カメラ内部パラメータ
K = np.array([
    [500, 0, 320],
    [0, 500, 240],
    [0, 0, 1]
])

# カメラ1：原点、正面を向く
R1 = np.eye(3)
t1 = np.zeros(3)
P1 = create_projection_matrix(K, R1, t1)

# カメラ2：右に移動、少し内向き
theta = np.radians(10)
R2 = np.array([
    [np.cos(theta), 0, np.sin(theta)],
    [0, 1, 0],
    [-np.sin(theta), 0, np.cos(theta)]
])
t2 = np.array([1, 0, 0])  # 右へ1m移動
P2 = create_projection_matrix(K, R2, t2)

# 真の3D点
X_true = np.array([0.5, 0.3, 3.0])

# 各カメラへの投影
x1 = project_point(P1, X_true)
x2 = project_point(P2, X_true)

print("=== ノイズなし ===")
print(f"真の3D点: {X_true}")
print(f"カメラ1への投影: ({x1[0]:.2f}, {x1[1]:.2f})")
print(f"カメラ2への投影: ({x2[0]:.2f}, {x2[1]:.2f})")

# DLTによる三角測量
X_estimated = triangulate_dlt([x1, x2], [P1, P2])

print(f"\n推定された3D点: {X_estimated}")
print(f"誤差: {np.linalg.norm(X_estimated - X_true):.6f}")

In [ ]:
# ノイズありのテスト
def test_triangulation_with_noise(noise_levels: List[float]):
    """異なるノイズレベルでの三角測量精度を評価"""
    
    n_trials = 100
    errors = {level: [] for level in noise_levels}
    
    for level in noise_levels:
        for _ in range(n_trials):
            # ノイズを加えた観測点
            x1_noisy = x1 + np.random.randn(2) * level
            x2_noisy = x2 + np.random.randn(2) * level
            
            # 三角測量
            X_est = triangulate_dlt([x1_noisy, x2_noisy], [P1, P2])
            
            # 誤差
            error = np.linalg.norm(X_est - X_true)
            errors[level].append(error)
    
    return errors

noise_levels = [0.0, 0.5, 1.0, 2.0, 5.0]
errors = test_triangulation_with_noise(noise_levels)

# 結果の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# 箱ひげ図
box_data = [errors[level] for level in noise_levels]
ax1.boxplot(box_data, labels=[f'{level}px' for level in noise_levels])
ax1.set_xlabel('Noise Level (pixels)', fontsize=12)
ax1.set_ylabel('3D Error (meters)', fontsize=12)
ax1.set_title('DLT Triangulation Error vs Noise Level', fontsize=12)
ax1.grid(True, alpha=0.3)

# 平均誤差
mean_errors = [np.mean(errors[level]) for level in noise_levels]
std_errors = [np.std(errors[level]) for level in noise_levels]

ax2.errorbar(noise_levels, mean_errors, yerr=std_errors, 
             fmt='o-', capsize=5, capthick=2)
ax2.set_xlabel('Noise Level (pixels)', fontsize=12)
ax2.set_ylabel('Mean 3D Error (meters)', fontsize=12)
ax2.set_title('Mean Error with Standard Deviation', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nノイズレベルと平均誤差:")
for level, mean_err, std_err in zip(noise_levels, mean_errors, std_errors):
    print(f"  {level} pixels → {mean_err:.4f} ± {std_err:.4f} meters")

---

## 4. 中点法（Midpoint Method）

### 4.1 概要

2つの光線間の**最短距離の中点**を3D点の推定値とする幾何学的な手法。

### 4.2 アルゴリズム

光線1: $\mathbf{r}_1(s) = \mathbf{o}_1 + s \mathbf{d}_1$

光線2: $\mathbf{r}_2(t) = \mathbf{o}_2 + t \mathbf{d}_2$

最短距離を与える $s^*, t^*$ を求め：

$$\mathbf{X} = \frac{\mathbf{r}_1(s^*) + \mathbf{r}_2(t^*)}{2}$$

In [ ]:
def triangulate_midpoint(x1: np.ndarray, x2: np.ndarray,
                          P1: np.ndarray, P2: np.ndarray) -> np.ndarray:
    """中点法による三角測量
    
    Args:
        x1, x2: 各カメラでの2D点 (u, v)
        P1, P2: 投影行列 (3x4)
    
    Returns:
        X: 3D点の座標
    """
    # カメラ中心を計算
    # P = K[R|t] → C = -R^T @ t
    # または P の右零空間
    def get_camera_center(P):
        U, S, Vt = np.linalg.svd(P)
        C_h = Vt[-1]
        C = C_h[:3] / C_h[3]
        return C
    
    C1 = get_camera_center(P1)
    C2 = get_camera_center(P2)
    
    # 光線の方向を計算
    # 正規化座標を使用: K^{-1} @ x
    # 簡略化のため、Pの擬似逆行列を使用
    x1_h = np.array([x1[0], x1[1], 1])
    x2_h = np.array([x2[0], x2[1], 1])
    
    # Pの3x3部分の逆行列を使用して方向を計算
    M1 = P1[:, :3]
    M2 = P2[:, :3]
    
    d1 = np.linalg.inv(M1) @ x1_h
    d1 = d1 / np.linalg.norm(d1)
    
    d2 = np.linalg.inv(M2) @ x2_h
    d2 = d2 / np.linalg.norm(d2)
    
    # 最短距離の計算
    w0 = C1 - C2
    a = np.dot(d1, d1)
    b = np.dot(d1, d2)
    c = np.dot(d2, d2)
    d = np.dot(d1, w0)
    e = np.dot(d2, w0)
    
    denom = a * c - b * b
    
    if abs(denom) < 1e-10:
        # 光線が平行な場合
        return (C1 + C2) / 2
    
    s = (b * e - c * d) / denom
    t = (a * e - b * d) / denom
    
    # 各光線上の最近点
    point1 = C1 + s * d1
    point2 = C2 + t * d2
    
    # 中点
    X = (point1 + point2) / 2
    
    return X

# テスト
print("=== 中点法のテスト ===")
X_midpoint = triangulate_midpoint(x1, x2, P1, P2)
print(f"真の3D点: {X_true}")
print(f"中点法による推定: {X_midpoint}")
print(f"誤差: {np.linalg.norm(X_midpoint - X_true):.6f}")

# ノイズあり
x1_noisy = x1 + np.random.randn(2) * 1.0
x2_noisy = x2 + np.random.randn(2) * 1.0

X_midpoint_noisy = triangulate_midpoint(x1_noisy, x2_noisy, P1, P2)
X_dlt_noisy = triangulate_dlt([x1_noisy, x2_noisy], [P1, P2])

print(f"\nノイズあり (1 pixel):")
print(f"  中点法誤差: {np.linalg.norm(X_midpoint_noisy - X_true):.4f}")
print(f"  DLT誤差: {np.linalg.norm(X_dlt_noisy - X_true):.4f}")

---

## 5. 最適三角測量（Optimal Triangulation）

### 5.1 問題点

DLTや中点法は、観測点のノイズを考慮していません。

### 5.2 最適三角測量のアイデア

観測点 $\mathbf{x}_i$ を**エピポーラ線上に補正**し、エピポーラ拘束を満たすようにしてから三角測量を行います。

### 5.3 最小化問題

$$\min_{\hat{\mathbf{x}}_1, \hat{\mathbf{x}}_2} \|\mathbf{x}_1 - \hat{\mathbf{x}}_1\|^2 + \|\mathbf{x}_2 - \hat{\mathbf{x}}_2\|^2$$

subject to: $\hat{\mathbf{x}}_2^\top \mathbf{F} \hat{\mathbf{x}}_1 = 0$

### 5.4 非線形最適化によるアプローチ

実際には、**再投影誤差の最小化**として定式化することが多いです：

$$\min_{\mathbf{X}} \sum_i \|\mathbf{x}_i - \pi(\mathbf{P}_i, \mathbf{X})\|^2$$

In [ ]:
def triangulate_optimal(points_2d: List[np.ndarray], 
                        proj_matrices: List[np.ndarray],
                        initial_X: Optional[np.ndarray] = None) -> np.ndarray:
    """非線形最適化による三角測量（再投影誤差最小化）
    
    Args:
        points_2d: 各カメラでの2D点のリスト
        proj_matrices: 各カメラの投影行列のリスト
        initial_X: 初期値（Noneの場合はDLTで計算）
    
    Returns:
        X: 最適化された3D点の座標
    """
    # 初期値
    if initial_X is None:
        initial_X = triangulate_dlt(points_2d, proj_matrices)
    
    def reprojection_error(X):
        """再投影誤差を計算"""
        errors = []
        X_h = np.append(X, 1)
        
        for pt, P in zip(points_2d, proj_matrices):
            x_proj_h = P @ X_h
            x_proj = x_proj_h[:2] / x_proj_h[2]
            
            errors.extend(pt - x_proj)
        
        return np.array(errors)
    
    # Levenberg-Marquardt法で最適化
    result = least_squares(reprojection_error, initial_X, method='lm')
    
    return result.x

# テスト
print("=== 最適三角測量のテスト ===")

# ノイズを加えた観測点
np.random.seed(42)
noise_level = 2.0
x1_noisy = x1 + np.random.randn(2) * noise_level
x2_noisy = x2 + np.random.randn(2) * noise_level

# 各手法で推定
X_dlt = triangulate_dlt([x1_noisy, x2_noisy], [P1, P2])
X_optimal = triangulate_optimal([x1_noisy, x2_noisy], [P1, P2])

print(f"真の3D点: {X_true}")
print(f"DLT: {X_dlt}, 誤差: {np.linalg.norm(X_dlt - X_true):.4f}")
print(f"最適化: {X_optimal}, 誤差: {np.linalg.norm(X_optimal - X_true):.4f}")

# 再投影誤差の比較
def compute_reprojection_error(X, points_2d, proj_matrices):
    total_error = 0
    X_h = np.append(X, 1)
    for pt, P in zip(points_2d, proj_matrices):
        x_proj_h = P @ X_h
        x_proj = x_proj_h[:2] / x_proj_h[2]
        total_error += np.linalg.norm(pt - x_proj) ** 2
    return np.sqrt(total_error / len(points_2d))

reproj_dlt = compute_reprojection_error(X_dlt, [x1_noisy, x2_noisy], [P1, P2])
reproj_optimal = compute_reprojection_error(X_optimal, [x1_noisy, x2_noisy], [P1, P2])

print(f"\n再投影誤差 (RMS):")
print(f"  DLT: {reproj_dlt:.4f} pixels")
print(f"  最適化: {reproj_optimal:.4f} pixels")

---

## 6. 多視点三角測量

### 6.1 2視点を超える利点

- 冗長な観測によりノイズの影響を軽減
- より頑健な推定が可能
- オクルージョンへの対応

### 6.2 実装

In [ ]:
def setup_multiview_cameras(n_cameras: int, radius: float = 2.0,
                             K: np.ndarray = None) -> List[np.ndarray]:
    """円周上に配置されたカメラの投影行列を生成"""
    if K is None:
        K = np.array([
            [500, 0, 320],
            [0, 500, 240],
            [0, 0, 1]
        ])
    
    proj_matrices = []
    camera_centers = []
    
    for i in range(n_cameras):
        angle = 2 * np.pi * i / n_cameras
        
        # カメラ位置（円周上）
        C = np.array([radius * np.cos(angle), 0, radius * np.sin(angle)])
        camera_centers.append(C)
        
        # 原点を向くように回転
        # Z軸が原点を向く
        z_axis = -C / np.linalg.norm(C)
        y_axis = np.array([0, 1, 0])
        x_axis = np.cross(y_axis, z_axis)
        x_axis = x_axis / np.linalg.norm(x_axis)
        y_axis = np.cross(z_axis, x_axis)
        
        R = np.vstack([x_axis, y_axis, z_axis])
        t = -R @ C
        
        P = create_projection_matrix(K, R, t)
        proj_matrices.append(P)
    
    return proj_matrices, camera_centers

def test_multiview_triangulation():
    """多視点三角測量のテスト"""
    # 真の3D点
    X_true = np.array([0.0, 0.0, 0.0])
    
    # カメラ数を変えてテスト
    camera_counts = [2, 3, 4, 6, 8]
    noise_level = 2.0
    n_trials = 50
    
    results = {n: [] for n in camera_counts}
    
    for n_cameras in camera_counts:
        proj_matrices, _ = setup_multiview_cameras(n_cameras)
        
        for _ in range(n_trials):
            # 各カメラへの投影（ノイズ付き）
            points_2d = []
            for P in proj_matrices:
                x = project_point(P, X_true)
                x_noisy = x + np.random.randn(2) * noise_level
                points_2d.append(x_noisy)
            
            # 三角測量
            X_est = triangulate_dlt(points_2d, proj_matrices)
            error = np.linalg.norm(X_est - X_true)
            results[n_cameras].append(error)
    
    return results

# テストの実行
print("多視点三角測量のテスト中...")
multiview_results = test_multiview_triangulation()
print("完了!")

# 結果の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

camera_counts = list(multiview_results.keys())
mean_errors = [np.mean(multiview_results[n]) for n in camera_counts]
std_errors = [np.std(multiview_results[n]) for n in camera_counts]

# 箱ひげ図
ax1.boxplot([multiview_results[n] for n in camera_counts], 
            labels=[str(n) for n in camera_counts])
ax1.set_xlabel('Number of Cameras', fontsize=12)
ax1.set_ylabel('3D Error (meters)', fontsize=12)
ax1.set_title('Multi-view Triangulation: Error vs Camera Count', fontsize=12)
ax1.grid(True, alpha=0.3)

# 平均誤差
ax2.errorbar(camera_counts, mean_errors, yerr=std_errors,
             fmt='o-', capsize=5, capthick=2, markersize=8)
ax2.set_xlabel('Number of Cameras', fontsize=12)
ax2.set_ylabel('Mean 3D Error (meters)', fontsize=12)
ax2.set_title('Mean Error Decreases with More Views', fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nカメラ数と平均誤差:")
for n, mean_err, std_err in zip(camera_counts, mean_errors, std_errors):
    print(f"  {n} cameras → {mean_err:.4f} ± {std_err:.4f} meters")

In [ ]:
def visualize_multiview_setup():
    """多視点カメラ配置の可視化"""
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    n_cameras = 6
    proj_matrices, camera_centers = setup_multiview_cameras(n_cameras)
    
    # 3D点
    X_points = np.array([
        [0, 0, 0],
        [0.3, 0.2, 0.1],
        [-0.2, 0.1, -0.1],
        [0.1, -0.3, 0.2]
    ])
    
    colors = plt.cm.tab10(np.linspace(0, 1, n_cameras))
    
    # カメラの描画
    for i, (C, P, color) in enumerate(zip(camera_centers, proj_matrices, colors)):
        ax.scatter(*C, color=color, s=200, marker='o', label=f'Camera {i+1}')
        
        # カメラの向き（光軸）
        direction = -C / np.linalg.norm(C)
        ax.quiver(*C, *direction * 0.5, color=color, arrow_length_ratio=0.2)
    
    # 3D点の描画
    for X in X_points:
        ax.scatter(*X, color='red', s=150, marker='*')
        
        # 各カメラからの光線
        for C, color in zip(camera_centers, colors):
            ax.plot([C[0], X[0]], [C[1], X[1]], [C[2], X[2]], 
                    color=color, alpha=0.3, linewidth=1)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Multi-view Camera Setup ({n_cameras} Cameras)', fontsize=14)
    ax.legend(loc='upper left')
    
    # 等軸スケール
    max_range = 2.5
    ax.set_xlim(-max_range, max_range)
    ax.set_ylim(-max_range, max_range)
    ax.set_zlim(-max_range, max_range)
    
    ax.view_init(elev=30, azim=45)
    
    plt.tight_layout()
    plt.show()

visualize_multiview_setup()

---

## 7. 三角測量の誤差解析

### 7.1 誤差要因

| 要因 | 説明 | 対策 |
|------|------|------|
| **画像ノイズ** | 特徴点検出の不確実性 | サブピクセル精度、多視点 |
| **キャリブレーション誤差** | カメラパラメータの不正確さ | 高精度キャリブレーション |
| **ベースライン** | 短すぎると深度精度が低下 | 適切なベースライン設計 |
| **視差角** | 小さい角度では精度が低下 | 三角測量角度の確保 |

### 7.2 深度誤差とベースラインの関係

深度誤差 $\delta Z$ と視差誤差 $\delta d$ の関係：

$$\delta Z = \frac{Z^2}{f \cdot b} \delta d$$

- 深度が大きいほど誤差が増大（$Z^2$ に比例）
- ベースライン $b$ が大きいほど精度向上

In [ ]:
def analyze_depth_error():
    """深度誤差の解析"""
    f = 500  # 焦点距離（ピクセル）
    baselines = [0.1, 0.2, 0.5, 1.0]  # ベースライン（メートル）
    delta_d = 1.0  # 視差誤差（ピクセル）
    
    depths = np.linspace(1, 20, 100)  # 深度範囲
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(baselines)))
    
    for b, color in zip(baselines, colors):
        # 深度誤差: δZ = Z² / (f * b) * δd
        depth_errors = (depths ** 2) / (f * b) * delta_d
        
        ax1.plot(depths, depth_errors, color=color, linewidth=2, 
                 label=f'b = {b*100:.0f} cm')
    
    ax1.set_xlabel('Depth Z (meters)', fontsize=12)
    ax1.set_ylabel('Depth Error δZ (meters)', fontsize=12)
    ax1.set_title(f'Depth Error vs Distance (δd = {delta_d} pixel)', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(1, 20)
    ax1.set_ylim(0, 2)
    
    # 相対誤差
    for b, color in zip(baselines, colors):
        relative_errors = depths / (f * b) * delta_d * 100  # パーセント
        
        ax2.plot(depths, relative_errors, color=color, linewidth=2,
                 label=f'b = {b*100:.0f} cm')
    
    ax2.set_xlabel('Depth Z (meters)', fontsize=12)
    ax2.set_ylabel('Relative Error (%)', fontsize=12)
    ax2.set_title('Relative Depth Error (δZ/Z × 100)', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_xlim(1, 20)
    
    plt.tight_layout()
    plt.show()
    
    print("重要な洞察:")
    print("- 深度誤差は距離の2乗に比例して増大")
    print("- ベースラインを大きくすると深度精度が向上")
    print("- 相対誤差は距離に比例（一定の割合で増加）")

analyze_depth_error()

In [ ]:
def analyze_triangulation_angle():
    """三角測量角度と精度の関係"""
    # 固定の3D点
    X_true = np.array([0, 0, 5])
    
    # 三角測量角度（カメラ間の角度）を変化させる
    angles = np.linspace(5, 90, 18)  # 5度から90度
    
    K = np.array([
        [500, 0, 320],
        [0, 500, 240],
        [0, 0, 1]
    ])
    
    noise_level = 1.0
    n_trials = 100
    
    mean_errors = []
    std_errors = []
    
    for angle in angles:
        # 2つのカメラを角度に応じて配置
        half_angle = np.radians(angle / 2)
        distance = 5  # カメラから3D点までの距離
        
        # カメラ1
        C1 = np.array([-distance * np.tan(half_angle), 0, 0])
        R1 = np.eye(3)
        t1 = -R1 @ C1
        P1 = create_projection_matrix(K, R1, t1)
        
        # カメラ2
        C2 = np.array([distance * np.tan(half_angle), 0, 0])
        R2 = np.eye(3)
        t2 = -R2 @ C2
        P2 = create_projection_matrix(K, R2, t2)
        
        # 投影
        x1 = project_point(P1, X_true)
        x2 = project_point(P2, X_true)
        
        errors = []
        for _ in range(n_trials):
            x1_noisy = x1 + np.random.randn(2) * noise_level
            x2_noisy = x2 + np.random.randn(2) * noise_level
            
            X_est = triangulate_dlt([x1_noisy, x2_noisy], [P1, P2])
            error = np.linalg.norm(X_est - X_true)
            errors.append(error)
        
        mean_errors.append(np.mean(errors))
        std_errors.append(np.std(errors))
    
    # 可視化
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.errorbar(angles, mean_errors, yerr=std_errors,
                fmt='o-', capsize=3, capthick=1, markersize=6)
    ax.set_xlabel('Triangulation Angle (degrees)', fontsize=12)
    ax.set_ylabel('3D Error (meters)', fontsize=12)
    ax.set_title('Triangulation Error vs Viewing Angle', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    # 最適角度をマーク
    min_idx = np.argmin(mean_errors)
    ax.axvline(x=angles[min_idx], color='red', linestyle='--', 
               label=f'Optimal: {angles[min_idx]:.0f}°')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
    
    print(f"最小誤差の角度: {angles[min_idx]:.0f}度")
    print("\n洞察:")
    print("- 角度が小さすぎると（狭いベースライン）深度精度が悪化")
    print("- 角度が大きすぎると対応点マッチングが困難に")
    print("- 実用上は20-60度程度が推奨")

analyze_triangulation_angle()

---

## 8. 3D復元のワークフロー

### 8.1 完全な3D復元パイプライン

```
1. 特徴点検出・マッチング
       ↓
2. 基礎行列/本質行列の推定
       ↓
3. カメラ姿勢の復元（R, t）
       ↓
4. 三角測量による3D点の復元
       ↓
5. バンドル調整による最適化
```

In [ ]:
def full_3d_reconstruction_demo():
    """3D復元の完全なデモ"""
    print("=== 3D復元パイプラインのデモ ===")
    
    # 1. シーンの設定：複数の3D点
    np.random.seed(42)
    n_points = 20
    X_true = np.random.randn(n_points, 3) * 0.5
    X_true[:, 2] += 3  # カメラ前方に配置
    
    print(f"1. 真の3D点: {n_points}点")
    
    # 2. カメラの設定
    K = np.array([
        [500, 0, 320],
        [0, 500, 240],
        [0, 0, 1]
    ])
    
    # カメラ1: 原点
    R1 = np.eye(3)
    t1 = np.zeros(3)
    P1 = create_projection_matrix(K, R1, t1)
    
    # カメラ2: 右に移動、少し回転
    theta = np.radians(15)
    R2 = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    t2 = np.array([1, 0, 0])
    P2 = create_projection_matrix(K, R2, t2)
    
    print("2. 2台のカメラを設定")
    
    # 3. 投影（ノイズ付き）
    noise_level = 0.5
    points_1 = []
    points_2 = []
    
    for X in X_true:
        x1 = project_point(P1, X) + np.random.randn(2) * noise_level
        x2 = project_point(P2, X) + np.random.randn(2) * noise_level
        points_1.append(x1)
        points_2.append(x2)
    
    points_1 = np.array(points_1)
    points_2 = np.array(points_2)
    
    print(f"3. 各カメラへの投影（ノイズ: {noise_level}px）")
    
    # 4. 三角測量による復元
    X_reconstructed = []
    for x1, x2 in zip(points_1, points_2):
        X = triangulate_optimal([x1, x2], [P1, P2])
        X_reconstructed.append(X)
    
    X_reconstructed = np.array(X_reconstructed)
    
    print("4. 三角測量による3D復元完了")
    
    # 5. 誤差評価
    errors = np.linalg.norm(X_reconstructed - X_true, axis=1)
    
    print(f"\n=== 結果 ===")
    print(f"平均3D誤差: {np.mean(errors):.4f} m")
    print(f"最大3D誤差: {np.max(errors):.4f} m")
    print(f"最小3D誤差: {np.min(errors):.4f} m")
    
    return X_true, X_reconstructed, P1, P2, points_1, points_2

X_true, X_reconstructed, P1, P2, pts1, pts2 = full_3d_reconstruction_demo()

In [ ]:
def visualize_reconstruction_result(X_true, X_reconstructed, P1, P2):
    """復元結果の可視化"""
    fig = plt.figure(figsize=(14, 6))
    
    # 3Dプロット
    ax1 = fig.add_subplot(121, projection='3d')
    
    # カメラ中心
    def get_camera_center(P):
        U, S, Vt = np.linalg.svd(P)
        C_h = Vt[-1]
        return C_h[:3] / C_h[3]
    
    C1 = get_camera_center(P1)
    C2 = get_camera_center(P2)
    
    ax1.scatter(*C1, color='blue', s=200, marker='o', label='Camera 1')
    ax1.scatter(*C2, color='green', s=200, marker='o', label='Camera 2')
    
    # 真の点
    ax1.scatter(X_true[:, 0], X_true[:, 1], X_true[:, 2], 
                c='red', s=50, marker='o', alpha=0.5, label='True 3D Points')
    
    # 復元された点
    ax1.scatter(X_reconstructed[:, 0], X_reconstructed[:, 1], X_reconstructed[:, 2],
                c='blue', s=50, marker='x', alpha=0.7, label='Reconstructed')
    
    # 誤差ベクトル
    for Xt, Xr in zip(X_true, X_reconstructed):
        ax1.plot([Xt[0], Xr[0]], [Xt[1], Xr[1]], [Xt[2], Xr[2]], 
                 'k-', alpha=0.3, linewidth=0.5)
    
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('3D Reconstruction Result', fontsize=12)
    ax1.legend()
    ax1.view_init(elev=20, azim=-60)
    
    # 誤差のヒストグラム
    ax2 = fig.add_subplot(122)
    errors = np.linalg.norm(X_reconstructed - X_true, axis=1)
    
    ax2.hist(errors, bins=20, edgecolor='black', alpha=0.7)
    ax2.axvline(x=np.mean(errors), color='red', linestyle='--', 
                label=f'Mean: {np.mean(errors):.4f}m')
    ax2.set_xlabel('3D Error (meters)', fontsize=12)
    ax2.set_ylabel('Count', fontsize=12)
    ax2.set_title('Distribution of Reconstruction Errors', fontsize=12)
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

visualize_reconstruction_result(X_true, X_reconstructed, P1, P2)

---

## 9. まとめと次のステップ

### 学んだこと

1. **三角測量の原理**: 複数視点からの光線の交点として3D点を復元
2. **DLT法**: 線形方程式系をSVDで解く基本的な手法
3. **中点法**: 光線間の最短距離の中点を使用
4. **最適三角測量**: 再投影誤差を最小化する非線形最適化
5. **多視点三角測量**: 冗長な観測により精度向上
6. **誤差解析**: ベースライン、視差角と精度の関係

### 重要な数式

| 概念 | 数式 |
|------|------|
| 投影方程式 | $\lambda \mathbf{x} = \mathbf{P} \mathbf{X}$ |
| DLTの拘束 | $u(\mathbf{p}_3^\top \mathbf{X}) - (\mathbf{p}_1^\top \mathbf{X}) = 0$ |
| 深度誤差 | $\delta Z = \frac{Z^2}{fb} \delta d$ |

### 次のノートブック

**58. 特徴点検出とマッチング**では：
- SIFT, SURF, ORB などの特徴点検出
- 特徴量記述子
- マッチングアルゴリズム
- 外れ値除去

---

## 10. 自己評価クイズ

以下の質問に答えて理解度を確認しましょう：

1. 理想的な三角測量と実際の三角測量の違いは何ですか？

2. DLT法で行列 $\mathbf{A}$ を構築する際、各カメラからいくつの方程式が得られますか？

3. 中点法と DLT法の違いを説明してください。

4. 最適三角測量が DLT より優れている点は何ですか？

5. カメラ数を増やすと三角測量の精度が向上する理由は？

6. 深度誤差 $\delta Z = \frac{Z^2}{fb} \delta d$ から、精度を向上させるにはどうすればよいですか？

7. 三角測量角度が小さすぎる場合と大きすぎる場合、それぞれどのような問題が生じますか？

In [ ]:
# クイズの解答（隠し）
def show_quiz_answers():
    answers = """
    === 自己評価クイズ解答 ===
    
    1. 理想と現実の違い:
       - 理想: 2つの光線が1点で正確に交わる
       - 現実: ノイズにより光線は交わらない（ねじれの位置）
       - そのため、最良近似を求める必要がある
    
    2. DLTの方程式数:
       - 各カメラから2つの方程式が得られる
       - n視点で 2n × 4 の行列Aを構築
       - 最低2視点（4方程式）で解が求まる
    
    3. 中点法 vs DLT:
       - 中点法: 幾何学的（光線間の最短距離の中点）
       - DLT: 代数的（同次線形方程式系をSVDで解く）
       - どちらも線形近似だが、アプローチが異なる
    
    4. 最適三角測量の利点:
       - 再投影誤差を明示的に最小化
       - 画像ノイズの影響を考慮
       - エピポーラ拘束を満たすように補正
       - より統計的に最適な解が得られる
    
    5. カメラ数増加の効果:
       - 冗長な観測により、ノイズの影響が平均化される
       - 方程式数が増え、過決定系になる
       - 最小二乗解がより安定する
    
    6. 精度向上の方法:
       - ベースライン b を大きくする
       - 視差誤差 δd を小さくする（高精度な特徴点検出）
       - 焦点距離 f を大きくする（望遠レンズ）
       - 近距離で測定する（Z を小さく）
    
    7. 三角測量角度の問題:
       - 小さすぎる場合: 
         - 深度方向の不確実性が大きい
         - 小さな画像ノイズが大きな深度誤差に
       - 大きすぎる場合:
         - 対応点マッチングが困難（見え方が大きく異なる）
         - オクルージョンが増加
       - 実用上は20-60度程度が推奨
    """
    print(answers)

# 解答を見るには以下のコメントを外して実行
# show_quiz_answers()

---

## ナビゲーション

- **前のノートブック**: [56. ステレオ視と視差](56_stereo_vision_disparity_v1.ipynb)
- **次のノートブック**: [58. 特徴点検出とマッチング](58_feature_detection_matching_v1.ipynb)
- **カリキュラム**: [CURRICULUM_UNIT_0.3.md](CURRICULUM_UNIT_0.3.md)